# Simple predictions with a trained model

In [1]:
# This needs to be imported because the model uses a custom classes that need to be registered
from src.sketch_detection_rcnn.roi_heads import SketchROIHeads  # noqa # pylint: disable=unused-import

INFO:2023-11-28 23:05:23 5639:5639 init.cpp:158] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


In [6]:
import os
from src.utils.utils_json import read_json

model_name = 'fcb_7'

filename = os.path.join("reports", f"{model_name}.json")
model_meta = read_json(filename)

if "model_name" not in model_meta:
    model_meta["model_name"] = model_meta


In [7]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_meta["config_file"])

# path to the model we just trained
cfg.MODEL.WEIGHTS = os.path.join("models", model_name, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [8]:
def count_classes(classes: list[int]) -> dict[int, int]:
    class_count = {}
    for key in classes:
        if key in class_count:
            class_count[key] += 1
        else:
            class_count[key] = 1

    return class_count

In [9]:
import cv2
from detectron2.utils.visualizer import Visualizer
from src.visualization.visualize import show_img

im = cv2.imread("./data/test.jpg")
outputs = predictor(im)

# remove the colors of unsegmented pixels. This option is only available for segmentation models
visualizer = Visualizer(im, scale=0.5)

print("Predictions")
pred_out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))
show_img(pred_out.get_image())

[ WARN:0@170.423] global loadsave.cpp:248 findDecoder imread_('./data/test.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'